# recommendMe

In [1]:
import vk
import pandas as pd
import random
import time
from sklearn.externals import joblib
from scipy.sparse import csr_matrix
from pathlib import Path
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm_notebook
import requests

## Получаем информацию о юзерах, у которых указано больше N любимых фильмов

In [2]:
def get_data(id, n, min_num_of_movies=3):
    
    movies = pd.Series()
    
    file = Path("id")
    if file.is_file():
        id = joblib.load("id")
        movies = joblib.load("movies")
    last_id = 0
    count = 0
    
    while movies.size < n:
        
        f = vk.get_friends(str(id))
        
        friends = []
        for x in f:
            friends.append(str(x))
        
        user_info = vk.get_users_info(friends)
        
        for friend in user_info.keys():
            if movies.size >= n:
                break
            
            try:
                if count == 100:
                    print("pause")
                    time.sleep(1)
                    count = 0
                else:
                    count += 1
                    
                

                if ("movies" in user_info[friend].keys() 
                    and len(user_info[friend]["movies"].split(",")) >= min_num_of_movies
                    and not friend in movies.keys()
                    and len(friends) >= 5):
                    if movies.shape[0] % 20 == 0:
                        joblib.dump(movies, filename="movies", compress=9)
                        joblib.dump(friend, filename="id", compress=9)
                    movies.set_value(friend, user_info[friend]["movies"].split(","))
                    print(friend)
            except:
                pass
        print("Index: " + str(movies.shape[0]))
        print("Id: " + str(id))
        print("Last id: " + str(last_id))
        if movies.size < n:
            last_id = id
            while True: 
                ind = random.randint(0, len(friends)-1)
                if (len(vk.get_friends(friends[ind])) >= 5
                    and last_id != friends[ind]):
                    id = friends[ind]
                    break
            
    return movies

## Получаем список уникальных фильмов

In [3]:
def get_unique_movies(data):
    unique_movies = {}
    uniq_movies_reverse = {}
    for movies in tqdm_notebook(data):
        for movie_raw in movies:
            movie = movie_raw.strip().lower()
            if len(movie) < 3:
                continue
            if movie not in unique_movies:
                unique_movies[movie] = len(unique_movies)
                uniq_movies_reverse[unique_movies[movie]] = movie
    return unique_movies, unique_movies_reverse

## Создаем матрицу

In [4]:
def construct_matrix(data, unique_movies):
    row = []
    column = []
    user_dict = {}
    count = 0
    for user in tqdm_notebook(data.keys()):
        for movie in data[user]:
            if movie.strip().lower() in unique_movies:
                column.append(unique_movies[movie.lower().strip()])
                row.append(count)

        user_dict[count] = user
        count += 1
    ones = [1 for x in range(len(row))]
    
    joblib.dump(row, filename="row", compress=9)
    joblib.dump(column, filename="column", compress=9)

    movies_matrix = csr_matrix((ones, (row, column)), \
                               shape=(len(data), len(unique_movies)))
    return movies_matrix

# Коллаборативная фильтрация

In [ ]:
#file = Path("movies")
#if file.is_file():
#    data = joblib.load("movies")
#else:
data = get_data(1, 150000, min_num_of_movies=2)

142892755
152459922
143823137
102497485
157154363
pause
pause
Index: 48725
Id: 105705855
Last id: 0
pause
24183484
138172346
41585884
3608355
22886220
147684925
pause
133048806
89231129
80914814
119912399
pause
131158344
102414627
pause
85640794
156059937
99184050
pause
94779953
45583757
89758939
97486780
135776661
133363708
pause
133831715
99550807
12586259
42819340
pause
142978082
72873732
143053802
133489304
30827521
70220873
pause
119562367
133455306
127450169
140324265
pause
72513863
124577491
114680698
pause
127785233
27630938
135064241
pause
88791605
142191399
182415650
117130781
94609401
pause
86058479
139096756
182340223
99136828
pause
119774851
67857727
136412447
16723315
90979844
134140552
135660396
pause
124393316
28641217
34197654
25821515
pause
216412840
41248216
35589844
68438631
88736662
152845368
63856273
pause
155097774
20073042
pause
108119553
137549891
132960895
14685731
134860803
pause
84717983
138200863
57756995
105705855
103850230
pause
140671993
86196418
1240002

In [9]:
unique_movies, reverse_unique_movies = get_unique_movies(data)

NameError: name 'data' is not defined

In [12]:
matrix = construct_matrix(data, unique_movies)

In [69]:
svd = TruncatedSVD(n_components=100)
svd.fit(matrix)

TruncatedSVD(algorithm='randomized', n_components=100, n_iter=5,
       random_state=None, tol=0.0)

In [82]:
n = 150

In [83]:
y = svd.inverse_transform(svd.transform(matrix[n].todense()))

In [84]:
data[n]

['От заката до рассвета',
 ' Четыре комнаты',
 ' Криминальное чтиво',
 ' Красотка']

In [85]:
for key, value in sorted(enumerate(y[0]), key=lambda x: -x[1])[:10]:
    print(reverse_unique_movies[key], value)

криминальное чтиво 0.621845261382
красотка 0.207618720828
большой куш 0.145457958308
кокаин 0.121490788789
от заката до рассвета 0.103229837498
бесславные ублюдки 0.0915110685018
шаг вперед 0.0868200444847
назад в будущее 0.0815241763867
брат 0.0812211152282
форрест гамп 0.0731212055936


In [45]:
l = y.argsort()[:10]

# Кластеризация юзеров 

## Классификация групп

## Векторизация юзеров

In [8]:
num_of_blocks = len(data) // 1000 + 1
stuff = len(data) % 1000
users_info = []
for i in tqdm_notebook(range(num_of_blocks)):
    if i == i - 1:
        users_info.append(vk.get_users_info(data.keys()[i*1000:i*1000 + stuff]))
    else:
        users_info.append(vk.get_users_info(data.keys()[i*1000:(i+1)*1000]))
joblib.dump(users_info, filename="users_info", compress=9)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/mdm/anaconda3/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/mdm/anaconda3/lib/python3.5/site-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/home/mdm/anaconda3/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



['users_info']

In [9]:
def translate_users(users):
    info = pd.DataFrame(columns=('Sex', 'Age', 'Country', 'Movie', 'Liked'))
    for user in tqdm_notebook(users.keys()):
        sex = users[user]['sex']
        if sex == 2:
            sex = 0
        
        age = float('NaN')
        if 'bdate' in users[user].keys():
            ages = users[user]['bdate'].split(".")
            if len(ages) == 3:
                age = 2016 - int(ages[2])
        
        country = float('NaN')
        if 'country' in users[user].keys():
            country = users[user]['country']
            
        movie_set = set()
        
        if "movies" in users[user].keys():
            for movie in users[user]['movies'].split(','):
                if  unique_movies[unique_movies == movie].size != 0:
                    movie_set.add(unique_movies[unique_movies == movie].index[0])
            set_len = len(movie_set)
            good_scores = [1 for x in range(set_len)]
            for i in range(3):
                movie_set.add(random.randint(0, unique_movies.size - 1))
            bad_scores = [0 for x in range(len(movie_set)-set_len)]
            scores = []
            good_scores.extend(bad_scores)
            scores.extend(good_scores)

            movie_set = list(movie_set)

            for ind in range(len(scores)):
                info.loc[len(info)] = [sex, age, country, movie_set[ind], scores[ind]]
    return info

In [12]:
info = pd.DataFrame(columns=('Sex', 'Age', 'Country', 'Movie', 'Liked'))
d=[]
for i in range(len(users_info)-len(d)):
    i += len(d)
    d.append(translate_users(users_info[i]))
    joblib.dump(d, filename="information", compress=9)

KeyboardInterrupt: 